In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import random
import numpy as np
import pandas as pd

In [3]:
all_lines = []
for line in open("/content/drive/MyDrive/MajorProject/DATASET/Gutenburg Poem Dataset/gutenberg-poetry-v001.ndjson"):
    all_lines.append(json.loads(line.strip()))

In [4]:
corpus = "\n".join([line['s'] for line in random.sample(all_lines, 1000)])

In [5]:
!pip install markovify
import markovify

     |████████████████████████████████| 235 kB 5.1 MB/s 
  Created wheel for markovify: filename=markovify-0.9.3-py3-none-any.whl size=18622 sha256=df28684e93a0a9595e8411615839c24a5ee9f183412a99a1c9d5063dbc21f67b
  Stored in directory: /root/.cache/pip/wheels/d9/f0/5b/748a27bdf2496bd4df51acb9442dae516efce507ff4849813e
Successfully built markovify


In [6]:
model = markovify.NewlineText(corpus)

In [7]:
for i in range(5):
    print()
    for i in range(random.randrange(1, 4)):
        print(model.make_short_sentence(30))


Yet he is on;
And from his nose;

From Rhegium, by the fire,
And all the Gods,

He would not trace--

If love for to display

Away--away from the Crusade!
Where thou shalt stand
Thine by thy threats.


In [8]:
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils

In [9]:
# Lowercase all text
text = corpus.lower()

chars = list(set(text))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

vocab_size = len(chars)
print('Vocabulary size: {}'.format(vocab_size))

Vocabulary size: 64


In [10]:
# Data preparation
X = [] # training array
Y = [] # target array

length = len(text)
seq_length = 100 # number of characters to consider before predicting a character

# Iterate over length of text and create sequences stored in X, true values stored in Y
# true values being which character would actually come after sequence stored in X
for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_indices[char] for char in sequence])
    Y.append(char_indices[label])

print('Number of sequences: {}'.format(len(X)))

Number of sequences: 39296


In [11]:
# Reshape dimensions
X_new = np.reshape(X, (len(X), seq_length, 1))
# Scale values
X_new = X_new/float(len(chars))
# One-hot encode Y to remove ordinal relationships
Y_new = np_utils.to_categorical(Y)

X_new.shape, Y_new.shape

((39296, 100, 1), (39296, 64))

In [13]:
model = Sequential()
# Add LSTM layer to compute output using 150 LSTM units
model.add(LSTM(150, input_shape = (X_new.shape[1], X_new.shape[2]), return_sequences = True))

# Add regularization layer to prevent overfitting.
# Dropout ignores randomly selected neurons during training ("dropped out").
# Ultimately, network becomes less sensitive to specific weights of neurons --> network is better at generalization.
model.add(Dropout(0.1))

model.add(Flatten())
# Dense layer with softmax activation function to approximate probability distribution of next best word
model.add(Dense(Y_new.shape[1], activation = 'softmax'))

# Compile model to configure learning process
# Categorical crossentropy: an example can only belong to one class
# Adam optimization algorithm updates a learning rate for each network weight iteratively as learning unfolds
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Use 1 epoch for sake of computational time
model.fit(X_new, Y_new, epochs = 1, verbose = 1)

1228/1228 [==============================] - 39s 31ms/step - loss: 3.0675


In [14]:
# Random start
start = np.random.randint(0, len(X)-1)
string_mapped = list(X[start])
full_string = [indices_char[value] for value in string_mapped]

# Generate text
for i in range(400):
    x = np.reshape(string_mapped, (1, len(string_mapped), 1))
    x = x / float(len(chars))
    
    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [indices_char[value] for value in string_mapped]
    full_string.append(indices_char[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
# Combine text
newtext = ''
for char in full_string:
    newtext = newtext + char

print(newtext)

s, went the prince on board,
hath from her chains arisen.
and narrow cell sprang forth (1) and sough                                                                                                                                                                                                                                                                                                                                                                                                                


In [15]:
import keras.utils as ku
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

# Lowercase all text
text = corpus.lower()
text = text.split('\n')

# Create Tokenizer object to convert words to sequences of integers
tokenizer = Tokenizer(num_words = None, filters = '#$%&()*+-<=>@[\\]^_`{|}~\t\n', lower = False)

# Train tokenizer to the texts
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1

# Convert list of strings into flat dataset of sequences of tokens
sequences = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Pad sequences to ensure equal lengths
max_seq_len = max([len(x) for x in sequences])
sequences = np.array(pad_sequences(sequences, maxlen = max_seq_len, padding = 'pre'))

# Create n-grams sequence predictors and labels
predictors, label = sequences[:, :-1], sequences[:, -1]
label = ku.np_utils.to_categorical(label, num_classes = total_words)

In [16]:
import pickle
pickle.dump(tokenizer, open("/content/drive/MyDrive/MajorProject/MODELS/poem_gen_start_seq_tokenizer_3.pkl", "wb"))

In [17]:
print(tokenizer)

In [ ]:
# Input layer takes sequence of words as input
input_len = max_seq_len - 1
model = Sequential()
model.add(Embedding(total_words, 10, input_length = input_len))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Use 100 epoch for efficacy
model.fit(predictors, label, epochs = 1000, verbose = 1) #730pm

Epoch 1/1000
195/195 [==============================] - 11s 28ms/step - loss: 7.5338
Epoch 2/1000
195/195 [==============================] - 5s 28ms/step - loss: 7.0834
Epoch 3/1000
195/195 [==============================] - 5s 27ms/step - loss: 7.0087
Epoch 4/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.9835
Epoch 5/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.9691
Epoch 6/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.9620
Epoch 7/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.9626
Epoch 8/1000
195/195 [==============================] - 5s 27ms/step - loss: 6.9498
Epoch 9/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.8085
Epoch 10/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.7120
Epoch 11/1000
195/195 [==============================] - 5s 28ms/step - loss: 6.6370
Epoch 12/1000
195/195 [==============================] - 5s 28ms/step - l

In [ ]:
model.save('/content/drive/MyDrive/MajorProject/MODELS/poem_gen_start_seq_MODEL_2_1000epochs_1.h5')

In [ ]:
pickle.dump(model, open('/content/drive/MyDrive/MajorProject/MODELS/poem_gen_start_seq_MODEL_2_1000epochs_1.pkl', 'wb'))

In [ ]:
# Function to generate line
def generate_line(text, next_words, max_seq_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen = max_seq_len - 1, padding = 'pre')
        predicted = model.predict(token_list, verbose = 0)
        classes_x=np.argmax(predicted,axis=1)
        
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == classes_x:
                output_word = word
                break
        text += ' ' + output_word
    return text

In [ ]:
generate_line("flower dancing", 15, max_seq_len, model)

In [ ]:
generate_line("dog runs on a surf", 15, max_seq_len, model)